# VtemMax system

This is the forwad model for a transmitter waveform given by the current as a function of time

In [1]:
import sys
import os
import numpy
import pyp223

## Forward problem 

### Model specification

In [2]:
# number of layers (inlcuding halfspace)
nlyr=2
# number of fiducials/stations
nstat=1
# layer restitvities for each fiducial (Ohm meters) do not include halfspace
res=numpy.array([300,1000])
# basement resistvity
pbres=1000.0
# layer thicknesses
thk=numpy.ones([(nlyr-1)*nstat])*25.0 
# number of thin plates
nplt=1
# plate location - easting
peast=numpy.array([50])
# plote location - northing
pnorth=numpy.array([25])
# plate location - depth
ptop=numpy.array([30])
# plate resistivity
pres=numpy.array([1.0])
# plate length 1
plngth1=numpy.array([100])
# plate length 2
plngth2=numpy.array([100])
# plate width 1
pwdth1=numpy.array([0.0])
# plate width 2
pwdth2=numpy.array([90])
# cell width
cellw = 25
# plate thickness
pthk=numpy.array([1])
# dip azimuth
pdzm=numpy.deg2rad(numpy.array([90]))
# dip
pdip=numpy.deg2rad(numpy.array([60]))
# plunge
plng=numpy.deg2rad(numpy.array([0]))

### Vtem system specification - current waveform

In [3]:
### Read gates and waveform
fh=open('LeroiAir.cfl')
lines=fh.readlines()
nsx=int(lines[2].split()[1])
nchnl=int(lines[2].split()[4])
swx=[]
waveform=[]

for i in range(nsx):
    fields=lines[3+i].split()
    swx.append(float(fields[0])/1000.0)
    waveform.append(fields[1])

topn=[]
tcls=[]
for i in range(nchnl):
    fields=lines[3+nsx+i].split()
    topn.append(float(fields[0])/1000.)
    tcls.append(float(fields[1])/1000.)


swx=numpy.array(swx)
waveform=numpy.array(waveform)
topn=numpy.array(topn)
tcls=numpy.array(tcls)


# number of components
ncmp=2
# active components
cmp=2
# number transmitter turns
ntrn=3
# transmitter area
txarea=531
# number of channels nchnl read from cfl
# time at which the channel opens topn read from cfl
# time at which the channel closes tcls read from cfl
# number of samples on waveform nsx read from cfl
# times for waveform swx read from cfl
# amplitude type AMPS 0
ampt=0
# amplitude  waveform read from cfl
# transimtter easting/x-pos
tx=numpy.array([225.0])
# transmitter northing/y-pos
ty=numpy.array([100.0])
# transimtter height
tz=numpy.array([105.0])
# transmitter azimuth
tazi=numpy.deg2rad(numpy.array([0.0]))
# transmitter inclination
tincl=numpy.deg2rad(numpy.array([6.]))
# receiver easting/x-pos
rx=numpy.array([225.])
# receiever northin/y-pos
ry=numpy.array([100.])
# receiver height/z-pos
rz=numpy.array([105.]) 
# transmiter receiver separation inline
trdx=numpy.array([0.])
# transmitter receiver separation crossline
trdy=numpy.array([0.])
# transmitter receiver separation vertical
trdz=numpy.array([0.])

### Model prediction

In [4]:
# response
xmodl=numpy.zeros([nchnl*ncmp])
# ijac - active elements of Jacobian
ijac=None
# jacobian
a=None
# lCounter for leroiair failures
leroiair_failure_count=0

In [5]:
leroiair=pyp223.LeroiAir()

In [6]:
prd=leroiair.formod_vtem_max_data(nlyr,nstat,res,pbres,thk,nplt,peast,pnorth,ptop,pres,
                                         plngth1,plngth2,pwdth1,pwdth2,pthk,cellw,
                                         pdzm,pdip,plng,
                                         ncmp,cmp,
                                          ntrn,txarea,
                                         nchnl,topn,tcls,
                                         nsx,swx,ampt,waveform,
                                         tx,ty,tz,tazi,tincl,
                                         rx,ry,rz,trdx,trdy,trdz,
                                         xmodl,
                                         leroiair_failure_count)
        

#### Difference in percentage between executable and wrapper

In [7]:
fh=open('LeroiAir.out')
lines=fh.readlines()
ref=numpy.zeros([nchnl,ncmp])

ref[:,0]=numpy.concatenate((numpy.array(lines[3388].split()[4:]),numpy.array(lines[3395].split()[4:])),axis=0)
ref[:,1]=numpy.concatenate((numpy.array(lines[3405].split()[4:]),numpy.array(lines[3412].split()[4:])),axis=0)

In [8]:
(prd-ref)/ref*100.0

array([[-8.57968288e-03,  1.91985028e-02],
       [ 6.87368104e-03, -6.60812973e-03],
       [-8.06896552e-03, -1.35842264e-02],
       [ 1.04547260e-02, -2.79356061e-02],
       [ 4.51092118e-04, -5.84653756e-03],
       [-1.04545455e-02,  5.14403292e-03],
       [-1.66890337e-02, -1.66965124e-03],
       [ 1.06276400e-03, -7.48852821e-03],
       [ 3.88632015e-04, -4.91962603e-03],
       [ 3.34711254e-03,  1.54378743e-02],
       [-1.56856325e-02, -2.14911626e-02],
       [ 2.54619069e-02,  3.92778302e-03],
       [-2.67134733e-02, -8.04582904e-05],
       [-2.15202421e-03, -4.17618471e-03],
       [-5.19970522e-03, -3.92193775e-03],
       [ 4.46457895e-03,  1.73888967e-02],
       [ 4.78687306e-03, -2.86437282e-02],
       [ 4.97616996e-03,  1.75840641e-02],
       [-2.44922769e-02, -4.41686631e-03],
       [-2.89171405e-02, -1.52047336e-03],
       [ 4.55698744e-02,  1.42747931e-03],
       [-5.51285282e-03,  1.26375409e-02],
       [ 1.99403585e-03, -3.67866248e-03],
       [ 4.

#### Difference in percentage between wrapper1 and wrapper2

In [9]:
from wrapper_test import *

prd2 = forward(forward.model_vector(true_model)).reshape((-1,2))
(prd-prd2)/prd*100.0

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])